<a href="https://colab.research.google.com/github/tauseefhashmi/DataEngineeringKafkaPipelineStocks/blob/main/MasterThesisProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import urllib
import requests
from bs4 import BeautifulSoup
import re
url="https://bbooks.info/b/w/1f7fb81b4cb7171fbba0702a6f1702d31c198c1f/wild-sands.pdf"
res=requests.get(url)
html_page=res.content


In [ ]:
soup=BeautifulSoup(html_page,features='html.parser')
text=soup.find_all(text=True)
lines=[line2 for line in text for line2 in line.strip().split('.') if line2]
lines

In [ ]:
!pip install word2vec

     |████████████████████████████████| 51kB 2.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for word2vec: filename=word2vec-0.11.1-cp37-none-any.whl size=156413 sha256=cc25e9932e5d20763c0351d60c6722ce402ec948260e056b9f9c78e826298167
  Stored in directory: /root/.cache/pip/wheels/f3/7c/ac/fcb6d867f806021c3730fd848970db988b1d0030b5d20c0e02
Successfully built word2vec


In [ ]:
!pip install nltk
!python -m nltk.downloader all
!pip install unidecode
!pip install sklearn
!pip install gensim
!pip install numpy


In [ ]:
##Pre-Processing
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode
import string
def pre_process(corpus):
    # convert input corpus to lower case.
    corpus = corpus.lower()
    # collecting a list of stop words from nltk and punctuation form
    # string class and create single array.
    stopset = stopwords.words('english') + list(string.punctuation)
    # remove stop words and punctuations from string.
    # word_tokenize is used to tokenize the input corpus in word tokens.
    corpus = " ".join([i for i in word_tokenize(corpus) if i not in stopset])
    # remove non-ascii characters
    corpus = unidecode(corpus)
    return corpus
pre_process("Sample of non ASCII: Ceñía. How to remove stopwords and punctuations?")


'sample non ascii cenia remove stopwords punctuations'

In [ ]:
##Lemmitization
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()
sentence = "The striped bats are hanging on their feet for best"
words = word_tokenize(sentence)
for w in words:
  print(w, " : ", lemmatizer.lemmatize(w))

The  :  The
striped  :  striped
bats  :  bat
are  :  are
hanging  :  hanging
on  :  on
their  :  their
feet  :  foot
for  :  for
best  :  best


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# sentence pair
corpus = ["A girl is styling her hair.", "A girl is brushing her hair."]
for c in range(len(corpus)):
    corpus[c] = pre_process(corpus[c])
# creating vocabulary using uni-gram and bi-gram
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf_vectorizer.fit(corpus)
feature_vectors = tfidf_vectorizer.transform(corpus)

In [ ]:
from gensim.models import Word2Vec
import numpy as np
# give a path of model to load function
word_emb_model = Word2Vec.load('word2vec.bin')


In [ ]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-06-01 21:58:52--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.44.182
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.44.182|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
from gensim.models import Word2Vec
import numpy as np
# give a path of model to load function
#word_emb_model = Word2Vec.load(EMBEDDING_FILE)
Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True,norm_only=True)


In [ ]:
from gensim.models import Word2Vec
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
# give a path of model to load function
#word_emb_model = Word2Vec.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)
word_emb_model = KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
from collections import Counter
import itertools

def map_word_frequency(document):
    return Counter(itertools.chain(*document))
    
def get_sif_feature_vectors(sentence1, sentence2, word_emb_model=word_emb_model):
    sentence1 = [token for token in sentence1.split() if token in word_emb_model.wv.vocab]
    sentence2 = [token for token in sentence2.split() if token in word_emb_model.wv.vocab]
    word_counts = map_word_frequency((sentence1 + sentence2))
    embedding_size = 300 # size of vectore in word embeddings
    a = 0.001
    sentence_set=[]
    for sentence in [sentence1, sentence2]:
        vs = np.zeros(embedding_size)
        sentence_length = len(sentence)
        for word in sentence:
            a_value = a / (a + word_counts[word]) # smooth inverse frequency, SIF
            vs = np.add(vs, np.multiply(a_value, word_emb_model.wv[word])) # vs += sif * word_vector
        vs = np.divide(vs, sentence_length) # weighted average
        sentence_set.append(vs)
    return sentence_set

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

## **PIPELINE**



In [ ]:
!pip install -U gensim

     |████████████████████████████████| 23.9MB 1.9MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
#python example to train doc2vec model (with or without pre-trained word embeddings)

import gensim.models as g
import logging

#doc2vec parameters
vector_size = 300
window_size = 15
min_count = 1
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100
dm = 0 #0 = dbow; 1 = dmpv
worker_count = 1 #number of parallel processes

#pretrained word embeddings
pretrained_emb = "toy_data/pretrained_word_embeddings.txt" #None if use without pretrained embeddings

#input corpus
train_corpus = "toy_data/train_docs.txt"

#output model
saved_path = "toy_data/model.bin"

#enable logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#train doc2vec model
docs = g.doc2vec.TaggedLineDocument(train_corpus)
model = g.Doc2Vec(docs, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1, pretrained_emb=pretrained_emb, iter=train_epoch)

#save model
model.save(saved_path)


TypeError: ignored

In [ ]:
#python example to infer document vectors from trained doc2vec model
import gensim.models as g
import codecs

#parameters
model="toy_data/model.bin"
test_docs="toy_data/test_docs.txt"
output_file="toy_data/test_vectors.txt"

#inference hyper-parameters
start_alpha=0.01
infer_epoch=1000

#load model
m = g.Doc2Vec.load(model)
test_docs = [ x.strip().split() for x in codecs.open(test_docs, "r", "utf-8").readlines() ]

#infer test vectors
output = open(output_file, "w")
for d in test_docs:
    output.write( " ".join([str(x) for x in m.infer_vector(d, alpha=start_alpha, steps=infer_epoch)]) + "\n" )
output.flush()
output.close()


### PIPELINE 2

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [ ]:
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("my_doc2vec_model")

model.save(fname)
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

In [ ]:
vector = model.infer_vector(["system", "response"])

###PIPELINE

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = ["I'd like an apple","An apple a day keeps the doctor away","Never compare an apple to an orange","I prefer scikit-learn to Orange","The scikit-learn docs are Orange and Blue"]                                                                                                                                                                                                   
vect = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
tfidf = vect.fit_transform(corpus)                                                                                                                                                                                                                       
pairwise_similarity = tfidf * tfidf.T
print(pairwise_similarity)
print(pairwise_similarity.toarray())

  (0, 2)	0.27056873300683837
  (0, 1)	0.17668795478716204
  (0, 0)	0.9999999999999998
  (1, 2)	0.1543943648960287
  (1, 0)	0.17668795478716204
  (1, 1)	0.9999999999999999
  (2, 1)	0.1543943648960287
  (2, 0)	0.27056873300683837
  (2, 4)	0.16815247007633355
  (2, 3)	0.1963564882520361
  (2, 2)	1.0
  (3, 2)	0.1963564882520361
  (3, 4)	0.5449975578692606
  (3, 3)	0.9999999999999999
  (4, 2)	0.16815247007633355
  (4, 3)	0.5449975578692606
  (4, 4)	1.0
[[1.         0.17668795 0.27056873 0.         0.        ]
 [0.17668795 1.         0.15439436 0.         0.        ]
 [0.27056873 0.15439436 1.         0.19635649 0.16815247]
 [0.         0.         0.19635649 1.         0.54499756]
 [0.         0.         0.16815247 0.54499756 1.        ]]


###PIPELINE 4

In [ ]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt') 
!pip install python-docx
from docx import Document
text_sample1=[]
document = Document('/content/sample_data/BullDog.docx')
for para in document.paragraphs:
    print(para.text)
    text_sample1.append(para.text)
text_sample2=[]
document = Document('/content/sample_data/GoldenRetriever.docx')
for para in document.paragraphs:
    print(para.text)
    text_sample2.append(para.text)
text_sample3=[]
document = Document('/content/sample_data/Labrador.docx')
for para in document.paragraphs:
    print(para.text)
    text_sample3.append(para.text)    




stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


print(cosine_sim('a little bird', 'a little bird'))
print(cosine_sim('a little bird', 'a little bird chirps'))
print(cosine_sim('a little bird', 'a big dog barks'))
"""print(cosine_sim(text_sample1, text_sample2))
print(cosine_sim(text_sample1, text_sample3))
print(cosine_sim(text_sample2, text_sample3))"""

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
According to the American Kennel Club (AKC), a Bulldog's disposition should be "equable and kind, resolute, and courageous (not vicious or aggressive), and demeanor should be pacifist and dignified. These attributes should be countenanced by the expression and behavior".
Breeders have worked to reduce/remove aggression from these dogs. Most have a friendly, patient, but stubborn nature. Bulldogs are recognized as excellent family pets because of their tendency to form strong bonds with children.
Generally, Bulldogs are known for getting along well with children, other dogs, and other pets.
The temperament of the Golden Retriever is a hallmark of the breed, and is described in the standard as "kindly, friendly and confident".Golden Retrievers make good family pets, particularly as they are patient with children.They are not "one-man dogs" and are generally equally amiable with 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

'print(cosine_sim(text_sample1, text_sample2))\nprint(cosine_sim(text_sample1, text_sample3))\nprint(cosine_sim(text_sample2, text_sample3))'

In [ ]:
print(text_sample1)


[]


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc1 = nlp(u'Hello hi there!')
doc2 = nlp(u'Hello hi there!')
doc3 = nlp(u'Hey whatsup?')

print (doc1.similarity(doc2))
print (doc2.similarity(doc3))
print (doc1.similarity(doc3))

1.0
0.5156783284503755
0.5156783284503755


/usr/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if

In [ ]:
!pip install python-docx
from docx import Document
text_sample1=[]
document = Document('/content/sample_data/BullDog.docx')
for para in document.paragraphs:
    print(para.text)
    text_sample1.append(para.text)


According to the American Kennel Club (AKC), a Bulldog's disposition should be "equable and kind, resolute, and courageous (not vicious or aggressive), and demeanor should be pacifist and dignified. These attributes should be countenanced by the expression and behavior".
Breeders have worked to reduce/remove aggression from these dogs. Most have a friendly, patient, but stubborn nature. Bulldogs are recognized as excellent family pets because of their tendency to form strong bonds with children.
Generally, Bulldogs are known for getting along well with children, other dogs, and other pets.
##THIS IS SECOND TIME ['According to the American Kennel Club (AKC), a Bulldog\'s disposition should be "equable and kind, resolute, and courageous (not vicious or aggressive), and demeanor should be pacifist and dignified. These attributes should be countenanced by the expression and behavior".', 'Breeders have worked to reduce/remove aggression from these dogs. Most have a friendly, patient, but st

###**SENT2VEC**

In [ ]:
!pip install git+https://github.com/epfml/sent2vec


  Cloning https://github.com/epfml/sent2vec to /tmp/pip-req-build-jrrcxxfe
  Running command git clone -q https://github.com/epfml/sent2vec /tmp/pip-req-build-jrrcxxfe
  Created wheel for sent2vec: filename=sent2vec-0.0.0-cp37-cp37m-linux_x86_64.whl size=1130632 sha256=1670193526f2691c3824679355bd17c26dd6bba4311ed5fd69d3af15d92a9117
  Stored in directory: /tmp/pip-ephem-wheel-cache-1ab46ndz/wheels/f5/1a/52/b5f36e8120688b3f026ac0cefe9c6544905753c51d8190ff17
Successfully built sent2vec


In [ ]:
!pip install cython


In [ ]:
import sent2vec
model = sent2vec.Sent2vecModel()
model.load_model('model.bin')
emb = model.embed_sentence("once upon a time .") 
embs = model.embed_sentences(["first sentence .", "another sentence"])


###Word2Vec

In [ ]:
#Install tabula
!pip install tabula-py

# Import the required Module
#import tabula
from tabula.io import read_pdf,convert_into
# Read a PDF File
df = read_pdf("/content/drive/MyDrive/GoogleCollabWork/wild-sands.pdf", pages='all')[0]
print(df)
# convert PDF into CSV
convert_into("/content/drive/MyDrive/GoogleCollabWork/wild-sands.pdf", "wildsand.csv", output_format="csv", pages='all')
print(df)


In [ ]:
!pip install spacy

In [ ]:
import pandas as pd
import os
import gensim
import spacy
import en_core_web_sm
#nlp = en_core_web_sm.load()
nlp = spacy.load('en_core_web_sm')
data=pd.read_csv('/content/drive/MyDrive/GoogleCollabWork/wild-sands.csv')


data.head()


,text
0,By Chris Mason
1,Copyright 2012 by Christopher A. Mason
2,This ebook is an authorized free edition from ...
3,"Although you do not have to pay for this book,..."
4,fully protected by international Copyright law...


In [ ]:
s = pd.Series([i for i in range(4082)])
data=data.set_index([s, s])
data=data.rename_axis("Id",axis='columns')
data.head()  

,Id,text
0,0,By Chris Mason
1,1,Copyright 2012 by Christopher A. Mason
2,2,This ebook is an authorized free edition from ...
3,3,"Although you do not have to pay for this book,..."
4,4,fully protected by international Copyright law...


In [ ]:
ref_sent=data.loc[446]['text']
print(ref_sent)

446    When he could see the water, there were often ...
Name: text, dtype: object


In [ ]:
ref_sent_vec=nlp('ref_sent')

In [ ]:
all_docs=[nlp(row) for row in data['text']]

In [ ]:
sims=[]
doc_id=[]
for i in range(len(all_docs)):
  sim=all_docs[i].similarity(ref_sent_vec)
  sims.append(sim)
  doc_id.append(i)
  sims_docs=pd.DataFrame(list(zip(doc_id,sims)),columns=['doc_id','sims'])

In [ ]:
sims_docs_sorted=sims_docs.sort_values(by="sims",ascending=False)

In [ ]:
top5_sims_docs=data.iloc[sims_docs_sorted['doc_id'][1:6]]

In [ ]:
#sims_docs_sorted=sims_docs_sorted['sims'][1:6].to_frame()
#sims_docs_sorted=sims_docs_sorted.to_frame()

In [ ]:
print(data.loc[446]['text'].values)

['When he could see the water, there were often huge pillars or pyramids of stone standing out']


In [ ]:
print(sims_docs_sorted)

In [ ]:
print(top5_sims_docs)
top5_sims_docs["Id"]=[13,4040,3130,3363,483]
print(top5_sims_docs)

Id                                                      text    Id
13   13                                          Chapter...    13
4040 4040                                         Chapte...  4040
3130 3130                                            Robin?”  3130
3363 3363                                            Robin?”  3363
483  483                                        little pond.   483
Id                                                      text    Id
13   13                                          Chapter...    13
4040 4040                                         Chapte...  4040
3130 3130                                            Robin?”  3130
3363 3363                                            Robin?”  3363
483  483                                        little pond.   483


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
hello=pd.DataFrame(sims_docs_sorted['sims'][1:6])

In [ ]:
print(hello)
hello['Id']=[13,4040,3130,3363,483]
print(hello)

          sims    Id
13    0.462434    13
4040  0.459097  4040
3130  0.454773  3130
3363  0.454773  3363
483   0.450695   483
          sims    Id
13    0.462434    13
4040  0.459097  4040
3130  0.454773  3130
3363  0.454773  3363
483   0.450695   483


In [ ]:
photo_df = top5_sims_docs.set_index('Id', drop=False)
print(photo_df)

Id                                                 text    Id
Id                                                           
13                                          Chapter...    13
4040                                         Chapte...  4040
3130                                            Robin?”  3130
3363                                            Robin?”  3363
483                                        little pond.   483
Id                                                 text    Id
Id                                                           
13                                          Chapter...    13
4040                                         Chapte...  4040
3130                                            Robin?”  3130
3363                                            Robin?”  3363
483                                        little pond.   483


In [ ]:
top_sim_scores=pd.merge(hello,top5_sims_docs,on="Id")


In [ ]:
print(top_sim_scores)

       sims    Id                                               text
0  0.462434    13                                        Chapter...
1  0.459097  4040                                         Chapte...
2  0.454773  3130                                            Robin?”
3  0.454773  3363                                            Robin?”
4  0.450695   483                                       little pond.


In [ ]:
for (text,sim) in zip(top_sim_scores['text'],top_sim_scores['sims']):
  print("The Top 5 similiar sentences are: {}\n with similiarity score as {:.2f}\n".format(text,sim))

The Top 5 similiar sentences are:                                       Chapter 1: Prelude
 with similiarity score as 0.46

The Top 5 similiar sentences are:                                        Chapter 13: Coda
 with similiarity score as 0.46

The Top 5 similiar sentences are: Robin?”
 with similiarity score as 0.45

The Top 5 similiar sentences are: Robin?”
 with similiarity score as 0.45

The Top 5 similiar sentences are: little pond.
 with similiarity score as 0.45



### Embedding Using Seq2Learner Website

In [ ]:
!pip install seqlearner
!pip install keras
!pip install tensorflow
!pip install pomegranate
!pip install matplotlib
!pip install numpy

In [ ]:
import pandas as pd
from seqlearner import Freq2Vec
sequences = pd.read_csv("/content/drive/MyDrive/GoogleCollabWork/harry-potter-the-chamber-secrets.csv", header=None)
freq2vec = Freq2Vec(sequences, word_length=3, window_size=5, emb_dim=25, loss="mean_squared_error", epochs=250)
freq2vec.freq2vec_maker()

TypeError: ignored

### Sent2Vec Using Seq2Learner Website

In [ ]:
#!pip install WordEmbedder
!pip install git+https://github.com/EliHei/SeqLearn

  Cloning https://github.com/EliHei/SeqLearn to /tmp/pip-req-build-y9xx1q4u
  Running command git clone -q https://github.com/EliHei/SeqLearn /tmp/pip-req-build-y9xx1q4u
  Created wheel for seqlearner: filename=seqlearner-0.0.7-cp37-none-any.whl size=55079 sha256=7666f8c004875514a69850a855a171530a5f62d1fc897ca0d8c90d13ca22a5f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-zhyfg8id/wheels/e4/04/aa/60285a30369d74b5c0b3b0ecbb261627877be5bafe633f64d8
Successfully built seqlearner


In [ ]:
import pandas as pd
import seqlearner
#from seqlearner import Sent2Vec
sequences = pd.read_csv("/content/drive/MyDrive/GoogleCollabWork/harry-potter-the-chamber-secrets.csv", header=None)
sent2vec = seqlearner.Sent2Vec(sequences, word_length=3, emb_dim=25, epoch=100, lr=0.2, wordNgrams=5, loss="hs", neg=20, thread=10, t=0.0000005, dropoutK=2, bucket=4000000)

encoding = sent2vec.sent2vec_maker()

TypeError: ignored

###ANOTHER WEBSITE METHODS FOR SENT2VEC

In [ ]:
!pip install git+https://github.com/facebookresearch/fastText.git

  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-w4m7frum
  Running command git clone -q https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-w4m7frum
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3089820 sha256=3680a59922033999b9c71abb62432248260156b905002c2f1b5ad04dad14f610
  Stored in directory: /tmp/pip-ephem-wheel-cache-xc2be8g1/wheels/69/f8/19/7f0ab407c078795bc9f86e1f6381349254f86fd7d229902355
Successfully built fasttext


### Model from Website https://radimrehurek.com/gensim/models/word2vec.html

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(sentences=common_texts, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [ ]:
model = Word2Vec.load("word2vec.model")
model.train([["hello", "world"]], total_examples=1, epochs=1)

(0, 2)

In [ ]:
vector = model.wv['computer']  # get numpy vector of a word
sims = model.wv.most_similar('computer', topn=10)  # get other similar words

In [ ]:
from gensim.models import KeyedVectors

# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

vector = wv['computer']  # Get numpy vector of a word

In [ ]:
from gensim.test.utils import datapath

# Load a word2vec model stored in the C *text* format.
wv_from_text = KeyedVectors.load_word2vec_format(datapath('word2vec_pre_kv_c'), binary=False)
# Load a word2vec model stored in the C *binary* format.
wv_from_bin = KeyedVectors.load_word2vec_format(datapath("euclidean_vectors.bin"), binary=True)

In [ ]:
word_vectors = model.wv
del model

In [ ]:
from gensim.models import Phrases

# Train a bigram detector.
bigram_transformer = Phrases(common_texts)

# Apply the trained MWE detector to a corpus, using the result to train a Word2vec model.
model = Word2Vec(bigram_transformer[common_texts], min_count=1)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
import gensim.downloader
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))
['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']

# Download the "glove-twitter-25" embeddings
glove_vectors = gensim.downloader.load('glove-twitter-25')

# Use the downloaded vectors as usual:
glove_vectors.most_similar('twitter')
[('facebook', 0.948005199432373),
 ('tweet', 0.9403423070907593),
 ('fb', 0.9342358708381653),
 ('instagram', 0.9104824066162109),
 ('chat', 0.8964964747428894),
 ('hashtag', 0.8885937333106995),
 ('tweets', 0.8878158330917358),
 ('tl', 0.8778461217880249),
 ('link', 0.8778210878372192),
 ('internet', 0.8753897547721863)]

In [ ]:
from gensim.test.utils import datapath
from gensim.models.word2vec import LineSentence
sentences = LineSentence(datapath('lee_background.cor'))
for sentence in sentences:
    pass

In [ ]:
from gensim.models import Word2Vec
sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]
model = Word2Vec(sentences, min_count=1)

In [ ]:
from gensim.models import Word2Vec
sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]
model = Word2Vec(min_count=1)
model.build_vocab(sentences)  # prepare the model vocabulary
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)  # train word vectors

(1, 30)

### Doc2Vec

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [ ]:
from gensim.test.utils import get_tmpfile

fname = get_tmpfile("my_doc2vec_model")

model.save(fname)
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

In [ ]:
vector = model.infer_vector(["system", "response"])


[ 0.02928366  0.14237206 -0.02267986  0.08773571 -0.00779648  0.08778913
  0.14808035  0.11708865  0.03329771 -0.16866688 -0.06759381  0.07259941
 -0.135719    0.06634886  0.01646323  0.10016165 -0.03790294  0.0559139
 -0.00920132  0.19620982  0.15503085 -0.08860038 -0.07688845  0.16444044
  0.24925159 -0.02824616 -0.12013091  0.02220447 -0.05598956  0.08269601
  0.00604811  0.05485802  0.07325888  0.12196562 -0.11325276  0.08562492
  0.08239219  0.04570205 -0.15454614  0.01678174  0.01239824 -0.07281502
  0.08888572 -0.13090032 -0.1082538  -0.07066141  0.00935184  0.12933943
 -0.02474796 -0.02513623]


In [ ]:
model.dv['doc003']

AttributeError: ignored

In [ ]:
from gensim.test.utils import datapath
from gensim.models.doc2vec import TaggedLineDocument
for document in TaggedLineDocument(datapath("head500.noblanks.cor")):
    pass

In [ ]:
print(document)

TaggedDocument(['function', 'group', 'alcohol', 'molecul', 'carbon', 'atom', 'attach', 'carbon', 'atom', 'hydrogen', 'atom', 'chemistri', 'alcohol', 'organ', 'compound', 'hydroxyl', 'group', 'bound', 'carbon', 'atom', 'alkyl', 'substitut', 'alkyl', 'group', 'gener', 'formula', 'simpl', 'acycl', 'alcohol', 'layman', 'term', 'word', 'alcohol', 'arab', 'al-', 'refer', 'ethanol', 'grain', 'alcohol', 'older', 'spirit', 'wine', 'alcohol', 'beverag', 'ethanol', 'colorless', 'volatil', 'liquid', 'mild', 'odor', 'obtain', 'ferment', 'sugar', 'industri', 'commonli', 'obtain', 'ethylen', 'hydrat', 'reaction', 'ethylen', 'water', 'presenc', 'phosphor', 'acid', 'ethanol', 'ed', 'encyclopedia', 'chemic', 'technolog', 'ed', 'york', 'john', 'wilei', 'son', 'ethanol', 'wide', 'depress', 'world', 'thousand', 'year', 'sens', 'underli', 'term', 'alcohol', 'addict', 'alcohol', 'alcohol', 'clarifi', 'adject', 'isopropyl', 'alcohol', 'propan--ol', 'wood', 'alcohol', 'methyl', 'alcohol', 'methanol', 'suffix',

###  Document similarity queries

In [ ]:
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.similarities import Similarity

index_tmpfile = get_tmpfile("index")
query = [(1, 2), (6, 1), (7, 2)]

index = Similarity(index_tmpfile, common_corpus, num_features=len(common_dictionary))  # build the index
similarities = index[query]  # get similarities between the query and all index documents
print("Similarity score is",similarities)

Similarity score is [0.38490018 0.4082483  0.33333334 0.27216554 0.57735026 0.
 0.         0.         0.        ]


/usr/local/lib/python3.7/dist-packages/gensim/similarities/docsim.py:518: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = numpy.hstack(shard_results)


In [ ]:
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile

index_tmpfile = get_tmpfile("index")
batch_of_documents = common_corpus[:]  # only as example
index = Similarity(index_tmpfile, common_corpus, num_features=len(common_dictionary))  # build the index
# the batch is simply an iterable of documents, aka gensim corpus:
for similarities in index[batch_of_documents]:
    print(similarities)

[0.99999994 0.23570226 0.28867513 0.23570226 0.         0.
 0.         0.         0.        ]
[0.23570226 1.         0.4082483  0.33333334 0.70710677 0.
 0.         0.         0.23570226]
[0.28867513 0.4082483  1.         0.61237246 0.28867513 0.
 0.         0.         0.        ]
[0.23570226 0.33333334 0.61237246 1.         0.         0.
 0.         0.         0.        ]
[0.         0.70710677 0.28867513 0.         0.99999994 0.
 0.         0.         0.        ]
[0.         0.         0.         0.         0.         1.
 0.70710677 0.57735026 0.        ]
[0.         0.         0.         0.         0.         0.70710677
 0.99999994 0.81649655 0.40824828]
[0.         0.         0.         0.         0.         0.57735026
 0.81649655 0.99999994 0.6666666 ]
[0.         0.23570226 0.         0.         0.         0.
 0.40824828 0.6666666  0.99999994]


/usr/local/lib/python3.7/dist-packages/gensim/similarities/docsim.py:518: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = numpy.hstack(shard_results)


In [ ]:
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile

index_tmpfile = get_tmpfile("index")
index = Similarity(index_tmpfile, common_corpus, num_features=len(common_dictionary))  # build the index

for similarities in index:  # yield similarities of the 1st indexed document, then 2nd...
    pass

/usr/local/lib/python3.7/dist-packages/gensim/similarities/docsim.py:518: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = numpy.hstack(shard_results)


In [ ]:
from gensim.test.utils import common_corpus, common_dictionary
from gensim.similarities import MatrixSimilarity

query = [(1, 2), (5, 4)]
index = MatrixSimilarity(common_corpus, num_features=len(common_dictionary))
sims = index[query]

In [ ]:
from gensim.corpora.textcorpus import TextCorpus
from gensim.test.utils import datapath, get_tmpfile
from gensim.similarities import Similarity

corpus = TextCorpus(datapath('testcorpus.mm'))
index_temp = get_tmpfile("index")
index = Similarity(index_temp, corpus, num_features=400)  # create index

query = next(iter(corpus))
result = index[query]  # search similar to `query` in index

for sims in index[corpus]:  # if you have more query documents, you can submit them all at once, in a batch
    pass

# There is also a special syntax for when you need similarity of documents in the index
# to the index itself (i.e. queries=indexed documents themselves). This special syntax
# uses the faster, batch queries internally and **is ideal for all-vs-all pairwise similarities**:
for similarities in index:  # yield similarities of the 1st indexed document, then 2nd...
    pass

/usr/local/lib/python3.7/dist-packages/gensim/similarities/docsim.py:518: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = numpy.hstack(shard_results)


In [ ]:
from gensim.corpora.textcorpus import TextCorpus
from gensim.test.utils import datapath, get_tmpfile
from gensim.similarities import Similarity

temp_fname = get_tmpfile("index")
output_fname = get_tmpfile("saved_index")

corpus = TextCorpus(datapath('testcorpus.txt'))
index = Similarity(output_fname, corpus, num_features=400)

index.save(output_fname)
loaded_index = index.load(output_fname)

In [ ]:
from gensim.corpora.textcorpus import TextCorpus
from gensim.test.utils import datapath
from gensim.similarities import Similarity

corpus = TextCorpus(datapath('testcorpus.txt'))
index = Similarity('temp', corpus, num_features=400)
similarities = index.similarity_by_id(1)

/usr/local/lib/python3.7/dist-packages/gensim/similarities/docsim.py:518: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = numpy.hstack(shard_results)


In [ ]:
from gensim.test.utils import common_texts
from gensim.corpora import Dictionary
from gensim.models import Word2Vec
from gensim.similarities import SoftCosineSimilarity,SparseTermSimilarityMatrix
#from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import WordEmbeddingSimilarityIndex

model = Word2Vec(common_texts, vector_size=20, min_count=1)  # train word-vectors
termsim_index = WordEmbeddingSimilarityIndex(model.wv)
dictionary = Dictionary(common_texts)
bow_corpus = [dictionary.doc2bow(document) for document in common_texts]
similarity_matrix = SparseTermSimilarityMatrix(termsim_index, dictionary)  # construct similarity matrix
docsim_index = SoftCosineSimilarity(bow_corpus, similarity_matrix, num_best=10)

query = 'graph trees computer'.split()  # make a query
sims = docsim_index[dictionary.doc2bow(query)]  # calculate similarity of query to each doc from bow_corpus
print(sims)

### Tf-IDF MODELLING

In [ ]:
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

dataset = api.load("text8")
dct = Dictionary(dataset)  # fit dictionary
corpus = [dct.doc2bow(line) for line in dataset]  # convert corpus to BoW format

model = TfidfModel(corpus)  # fit model
vector = model[corpus[0]]  # apply model to the first corpus document

[==================================================] 100.0% 31.6/31.6MB downloaded


In [ ]:
print(vector)

[(1, 0.006704047545684609), (2, 0.0030255603220721273), (3, 0.003156168449586299), (4, 0.0036673470201144674), (5, 0.004575122435127926), (6, 0.0028052608258295926), (7, 0.004064820137019515), (8, 0.00014963587508918375), (9, 0.0007492665180478759), (10, 0.004142807322609117), (11, 0.004149816941645728), (12, 0.0077498817493309525), (13, 0.00656024165742503), (14, 0.003891486499758776), (15, 0.005476877392392166), (16, 0.0018233938817994433), (17, 0.0032209070754237084), (18, 0.0017737283389229173), (19, 0.0023373507198140124), (20, 0.003725514968930464), (21, 0.00590342512385848), (22, 0.003072401062545206), (23, 0.0006668171096292247), (24, 0.0017594266221832493), (25, 0.004202080158963513), (26, 0.002967397324595724), (27, 0.004709756138185673), (28, 0.0014819657487289912), (29, 0.0031562459553171694), (30, 0.0031999829254611097), (31, 0.001215574949729317), (32, 0.003843126241898761), (33, 0.006499414537896336), (34, 0.004546489373863172), (35, 0.003855345600331923), (36, 0.0046313

### NAIL BITTING

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import os
import gensim
# Set file names for train and test data
test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
lee_train_file = os.path.join(test_data_dir, 'lee_background.cor')
lee_test_file = os.path.join(test_data_dir, 'lee.cor')

import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

###############################################################################
# Let's take a look at the training corpus
#
print(train_corpus[:2])

###############################################################################
# And the testing corpus looks like this:
#
print(test_corpus[:2])

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
#gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
# Build a vocabulary
model.build_vocab(train_corpus)


#print(f"Word 'penalty' appeared {model.wv.get_vecattr('penalty', 'count')} times in the training corpus.")


model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)


vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print(vector)

ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])


import collections

counter = collections.Counter(ranks)
print(counter)

print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))




In [ ]:
# Pick a random document from the corpus and infer a vector from the model
import random
doc_id = random.randint(0, len(train_corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

# Testing the Model

# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))


# MY Corpus Based Similairty Model

### Creating Custom Corpus

In [ ]:
import os, os.path
path = os.path.expanduser('~/natural_language_toolkit_data')
if not os.path.exists(path):
   os.mkdir(path)
os.path.exists(path)
print(path)

/root/natural_language_toolkit_data


In [ ]:
import nltk.data
path in nltk.data.path
print(path)

/root/natural_language_toolkit_data


In [ ]:
import nltk.data
nltk.data.load('/root/natural_language_toolkit_data/wordfile.txt', format = 'raw')

### Training and Building the Model

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import os
import gensim
# Set file names for train and test data
test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
lee_train_file = os.path.join(test_data_dir, 'lee_background.cor')
lee_test_file = os.path.join(test_data_dir, 'lee.cor')

import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

###############################################################################
# Let's take a look at the training corpus
#
print(train_corpus[:2])

###############################################################################
# And the testing corpus looks like this:
#
print(test_corpus[:2])

#JUST DOC2VEC CUSTOMIZED

In [ ]:
import nltk
import gensim
import numpy as np
nltk.download('punkt')
from nltk.tokenize import word_tokenize,sent_tokenize

file_docs = []
#Put the File Path of your 1st Text Document
with open ('/content/drive/MyDrive/GoogleCollabWork/nst1.txt') as f:
    tokens = sent_tokenize(f.read())
    for line in tokens:
        file_docs.append(line)

print("Number of documents:",len(file_docs))

In [2]:
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in file_docs]

In [ ]:
print(gen_docs)

In [ ]:
dictionary = gensim.corpora.Dictionary(gen_docs)
print(dictionary.token2id)

In [6]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

In [ ]:
tf_idf = gensim.models.TfidfModel(corpus)
for doc in tf_idf[corpus]:
    print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])

In [8]:
sims = gensim.similarities.Similarity('/content',tf_idf[corpus],num_features=len(dictionary))

In [ ]:
file2_docs = []

#Put the File Path of your 2nd Text Document
with open ('/content/drive/MyDrive/GoogleCollabWork/nst2.txt') as f:
    tokens = sent_tokenize(f.read())
    for line in tokens:
        file2_docs.append(line)

print("Number of documents:",len(file2_docs))  
for line in file2_docs:
    query_doc = [w.lower() for w in word_tokenize(line)]
    query_doc_bow = dictionary.doc2bow(query_doc)

In [ ]:
query_doc_tf_idf = tf_idf[query_doc_bow]
print('Comparing Result:', sims[query_doc_tf_idf]) 

In [ ]:
sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
print(sum_of_sims)

In [19]:
percentage_of_similarity = round(float((sum_of_sims / len(file_docs)) * 100))
print(f'Average similarity float: {float(sum_of_sims / len(file_docs))}')
print(f'Average similarity percentage: {float(sum_of_sims / len(file_docs)) * 100}')
print(f'Average similarity rounded percentage: {percentage_of_similarity}')

Average similarity float: 0.09743563532829284
Average similarity percentage: 9.743563532829285
Average similarity rounded percentage: 10


In [ ]:
print(sims[query_doc_tf_idf*100],"%")

In [ ]:
avg_sims = [] # array of averages

# for line in query documents
for line in file2_docs:
        # tokenize words
        query_doc = [w.lower() for w in word_tokenize(line)]
        # create bag of words
        query_doc_bow = dictionary.doc2bow(query_doc)
        # find similarity for each document
        query_doc_tf_idf = tf_idf[query_doc_bow]
        # print (document_number, document_similarity)
        print('Comparing Result:', sims[query_doc_tf_idf]) 
        # calculate sum of similarities for each query doc
        sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
        # calculate average of similarity for each query doc
        avg = sum_of_sims / len(file_docs)
        # print average of similarity for each query doc
        print(f'avg: {sum_of_sims / len(file_docs)}')
        # add average values into array
        avg_sims.append(avg)  
# calculate total average
total_avg = np.sum(avg_sims, dtype=np.float)
# round the value and multiply by 100 to format it as percentage
percentage_of_similarity = round(float(total_avg) * 100)
# if percentage is greater than 100
# that means documents are almost same
if percentage_of_similarity >= 100:
  percentage_of_similarity = 100

#BERT

In [ ]:
!pip install sentence-transformers

In [ ]:
sentences = [
    "Three years later, the coffin was still full of Jello.",
    "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
    "The person box was packed with jelly many dozens of months later.",
    "Standing on one's head at job interviews forms a lasting impression.",
    "It took him a month to finish the meal.",
    "He found a leprechaun in his walnut shell."
]

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

# initialize dictionary that will contain tokenized sentences
tokens = {'input_ids': [], 'attention_mask': []}

for sentence in sentences:
    # tokenize sentence and append to dictionary lists
    new_tokens = tokenizer.encode_plus(sentence, max_length=128, truncation=True,
                                       padding='max_length', return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

# reformat list of tensors into single tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
tokens['input_ids'].shape

In [ ]:
outputs = model(**tokens)
outputs.keys()

embeddings = outputs.last_hidden_state
embeddings

In [ ]:
embeddings.shape

In [ ]:
attention_mask = tokens['attention_mask']
attention_mask.shape

In [ ]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape

In [ ]:
mask

In [ ]:
masked_embeddings = embeddings * mask
masked_embeddings.shape

In [ ]:
masked_embeddings

In [ ]:
summed = torch.sum(masked_embeddings, 1)
summed.shape

summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape
summed_mask

In [ ]:
mean_pooled = summed / summed_mask
mean_pooled
from sklearn.metrics.pairwise import cosine_similarity
# convert from PyTorch tensor to numpy array
mean_pooled = mean_pooled.detach().numpy()

# calculate
cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

array([[0.3308891 , 0.72192585, 0.17475495, 0.44709647, 0.55483633]],
      dtype=float32)

In [ ]:
from nltk.corpus import gutenberg as gt
nltk.download("gutenberg")

In [ ]:
print(gt.fileids()) 

In [ ]:
Emma = gt.raw("austen-emma.txt")
#print(Emma)
Emma=Emma.format(r"\r\n\\")
text_file=open('/content/drive/MyDrive/GoogleCollabWork/bs1.txt','w')
text_file=text_file.write(Emma)

In [ ]:
print(text_file)

In [ ]:
Persuasion = gt.raw("austen-persuasion.txt")
print(Persuasion)
Persuasion=Persuasion.format(r"\r\n\\")
text_file=open('/content/drive/MyDrive/GoogleCollabWork/bs2.txt','w')
text_file=text_file.write(Persuasion)


# *BM25*

###STYLE1


In [ ]:
!pip install rank_bm25
!pip install git+https://www.git@github.com/dorianbrown/rank_bm25.git

In [80]:
from rank_bm25 import BM25Okapi

corpus = file_docs

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
print(tokenized_corpus)

In [ ]:
query = ''.join(file2_docs)
tokenized_query = query.split(" ")
doc_scores = bm25.get_scores(tokenized_query)
print(doc_scores)

In [ ]:
bm25.get_top_n(tokenized_query, corpus, n=1)

In [ ]:
query = "In the men's rankings, Manish Kaushik is at the 18th spot in the 63kg weight category while Ashish Kumar and Satish Kumar are at the 9th spot in weight categories of 75kg and 91kg."
tokenized_query = query.split(" ")
doc_scores = bm25.get_scores(tokenized_query)
print(doc_scores)

In [ ]:
!pip install tokenizers==0.8.0-rc1
!pip install transformers

In [ ]:
import tokenize
import tokenizers
import nltk
nltk.download('punkt') # one time execution
import re

In [ ]:
import bm25Similarity

In [107]:
class TokenizedDocument(object):
    def __init__(self, text):
        self.sentences = []
        self.text = text
        self.preprocessed_split_indices = []
        # this is only optionally kept
        self.sentence_tokens_list = None
        
    def get_sentence_strings(self):
        sentence_strings = []
        for sentence in self.sentences:
            if len(sentence) <= 0:
                continue
            #print(sentence)
            # the 0th element is the start index, so let's get the start of the first and end of the last
            start_idx = sentence[0][0]
            end_idx = sentence[-1][1]
            
            sentence_string = self.text[start_idx : end_idx]
            sentence_strings.append(sentence_string)
        return sentence_strings
        
    def get_longest_sentence_string(self):
        sentence_strings = self.get_sentence_strings()
        longest_sentence_string = ''
        for sentence_string in sentence_strings:
            if len(sentence_string) > len(longest_sentence_string):
                longest_sentence_string = sentence_string
                
        return longest_sentence_string
        
    def get_longest_sentence_token_indices(self):
        sentences_tokens_sorted = self.sentences.copy()
        sentences_tokens_sorted.sort(key = len, reverse = True)
        return sentences_tokens_sorted

In [121]:
textData1 = file_docs
documents1 =  TokenizedDocument(textData1)

In [110]:
textData2 = file2_docs
documents2 = TokenizedDocument(textData2)

In [ ]:
similarities = bm25Similarity(documents1,documents2,'DocumentLengthCorrection',1);